In [1]:
import textwrap
from collections import Counter

import altair as alt
import pandas as pd

In [2]:
def format_percentage(row, col_name):
    count = int(row["counts"])
    percentage = row["percentage"] * 100
    w = "response"
    if count > 1:
        w += "s"
    return f"{percentage:.0f}% ({count} {w})"


def add_formatted_column(data):
    total = data["counts"].sum()
    data["percentage"] = data["counts"] / total
    data["formatted"] = data.apply(format_percentage, axis=1, col_name="Time")
    return data

In [3]:
def wrap(text, width):
    return textwrap.wrap(text, width)

In [4]:
def plot(data, title, order, output_file, subtitle=[]):
    c = Counter(data[question].dropna().tolist())
    options, counts = zip(*c.items())

    data_question = pd.DataFrame(
        {
            "options": options,
            "counts": counts,
        }
    )

    sort = None
    if order:
        sort = order

    _data = add_formatted_column(data_question)

    bars = (
        alt.Chart(
            _data,
            title=alt.Title(
                wrap(title, width=40),
                subtitle=subtitle,
            ),
        )
        .mark_bar()
        .encode(
            x=alt.X("percentage:Q", axis=alt.Axis(format=".0%", title=None)),
            y=alt.Y("options:N", axis=alt.Axis(title=None, labels=True), sort=sort),
            color=alt.Color("options:N", legend=None),
        )
    )

    text = (
        alt.Chart(_data)
        .mark_text(align="left", baseline="middle", dx=3)
        .encode(
            x=alt.X("percentage:Q"),
            y=alt.Y("options:N", sort=sort),
            text=alt.Text("formatted"),
        )
    )

    chart = bars + text

    # chart = chart.properties(width=600, height=250)

    chart = chart.configure_title(
        fontSize=15,
        subtitleFontSize=15,
        anchor="start",
        orient="bottom",
        offset=15,
        subtitleColor="gray",
    )

    chart.show()
    chart.save(output_file, scale_factor=2)

In [5]:
data = pd.read_csv("data.txt", sep=";")

In [6]:
question = (
    "In your estimate, how much time have you saved as a result of working with us?"
)
order = ["No time saved", "Minutes", "Hours", "Days"]

plot(data, question, order, "time-saved.png")

alt.LayerChart(...)

In [7]:
question = (
    "How likely is it that you would recommend our support to a friend or colleague?"
)

order = [10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]

plot(
    data,
    question,
    order,
    "recommending.png",
    subtitle="0 means definitely not. 10 means definitely yes.",
)

alt.LayerChart(...)